# 環境の操作

Azure Machine Learning ワークスペースで多くの実験を実行し、場合によっては、実験コードが実行される環境で必要な特定の Python パッケージを指定する必要がありました。このラボでは、環境について詳しく説明します。

## ワークスペースに接続する

まず、Azure ML SDK を使用してワークスペースに接続する必要があります。

> **注**: 前回の演習を完了してから Azure サブスクリプションとの認証済みセッションの有効期限が切れている場合は、再認証を求めるメッセージが表示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存した構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 実験用データを準備する

このラボでは、糖尿病患者の詳細を含むデータセットを使用します。次のセルを実行してこのデータセットを作成します (前のラボで作成した場合、コードは新しいバージョンを作成します)。

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # 糖尿病 CSV ファイルを /data にアップロードする
                       target_path='diabetes-data/', # データストアのフォルダー パスに入れる
                       overwrite=True, # 同じ名前の既存のファイルを置き換える
                       show_progress=True)

# データストア上のパスから表形式のデータセットを作成する (しばらく時間がかかる場合があります)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# 表形式のデータセットを登録する
tab_data_set = tab_data_set.register(workspace=ws, 
                           name='diabetes dataset',
                           description='diabetes data',
                           tags = {'format':'CSV'},
                           create_new_version=True)

print('Dataset ready.')

## トレーニング スクリプトを作成する

次の 2 つのセルを実行して作成します。
1. 新しい実験用のフォルダー
2. **SCIkit-learn を使用** してモデルをトレーニングし、**matplotlib** を使用して ROC 曲線をプロットするトレーニング スクリプト ファイル。

In [ ]:
import os

# 実験ファイル用フォルダーを作成する
experiment_folder = 'diabetes_training_logistic'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# ライブラリをインポートする
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# 正規化ハイパーパラメーターを設定する (スクリプトに引数として渡される)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# 実験実行コンテキストを取得する
run = Run.get_context()

# 糖尿病データを読み込む (入力データセットとして渡される)
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe()

# フィーチャーとラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# ロジスティック回帰モデルをトレーニングする
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# ROC 曲線をプロットする
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# 対角 50% ラインをプロットする
plt.plot([0, 1], [0, 1], 'k--')
# モデルによって達成された FPR と TPR をプロットする
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# 出力フォルダーに保存されたファイルは、自動的に実験レコードにアップロードされます
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## 環境を定義する

Azure Machine Learning で実験として Python スクリプトを実行すると、Conda 環境が作成され、スクリプトの実行コンテキストが定義されます。Azure Machine Learning には、多くの共通パッケージを含む既定の環境を提供します。これには、実験実行の操作に必要なライブラリを含む **azureml-defaults** パッケージ、**Pandas** や **numpy** などの一般なパッケージが含まれます。

また、**Conda** または **PIP** を使用して独自の環境を定義し、パッケージを追加して、実験が必要なすべてのライブラリにアクセスできるようにすることもできます。 

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

# 実験用 Python 環境を作成する
diabetes_env = Environment("diabetes-experiment-env")
diabetes_env.python.user_managed_dependencies = False # 依存関係を Azure ML に管理させる
diabetes_env.docker.enabled = True # ドッカー コンテナーを使用する

# パッケージの依存関係のセットを作成する (必要に応じて Conda または PIP)
diabetes_packages = CondaDependencies.create(conda_packages=['scikit-learn','ipykernel','matplotlib', 'pandas'],
                                          pip_packages=['azureml-sdk','pyarrow'])

# 環境に依存関係を追加する
diabetes_env.python.conda_dependencies = diabetes_packages

print(diabetes_env.name, 'defined.')

これで、Estimator (または RunConfig) に割り当てることで、実験用に環境を使用できます。

次のコードでは、作成した環境を汎用 Estimator に割り当て、実験を送信します。実験の実行中に、ウィジェットおよび **azureml_logs/60_control_log.txt** 出力ログで実行の詳細を確認すると、Conda 環境が構築されていることがわかります。

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment
from azureml.widgets import RunDetails

# スクリプト パラメーターを設定する
script_params = {
    '--regularization': 0.1
}

# トレーニング データセットを取得する
diabetes_ds = ws.datasets.get("diabetes dataset")

# Estimator を作成する
estimator = Estimator(source_directory=experiment_folder,
              inputs=[diabetes_ds.as_named_input('diabetes')], # データセットを入力として渡す
              script_params=script_params,
              compute_target = 'local',
              environment_definition = diabetes_env,
              entry_script='diabetes_training.py')

# 実験を作成する
experiment = Experiment(workspace = ws, name = 'diabetes-training')

# 実験を実行する
run = experiment.submit(config=estimator)
# 実行中に実行の詳細を表示する
RunDetails(run).show()
run.wait_for_completion()

実験では、必要なすべてのパッケージを含む環境が正常に使用されました - Azure Machine Learning Studio で実行された実験のメトリックと出力を表示するか、以下のコード (**Scikit-learn** を使用してトレーニングされたモデルや **matplotlib を**使用して生成された ROC チャート イメージを含む) を実行して表示できます。

In [ ]:
# 指標の記録を取得する
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## 環境を登録する

必要なパッケージを使用して環境を定義する手間が省けたので、ワークスペースに登録できます。

In [ ]:
# 環境を登録する
diabetes_env.register(workspace=ws)

環境は、最初に作成したときに割り当てた名前 (この場合、*diabetes-experiment-env*) で登録されていることに注意してください。

環境が登録されている場合、同じ要件を持つすべてのスクリプトに再利用できます。たとえば、別のアルゴリズムを使用して糖尿病モデルをトレーニングするフォルダーとスクリプトを作成してみましょう。

In [ ]:
import os

# 実験ファイル用フォルダーを作成する
experiment_folder = 'diabetes_training_tree'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# ライブラリをインポートする
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# 実験実行コンテキストを取得する
run = Run.get_context()

# 糖尿病データを読み込む (入力データセットとして渡される)
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe()

# フィーチャーとラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# デシジョン ツリー モデルをトレーニングする
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# ROC 曲線をプロットする
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# 対角 50% ラインをプロットする
plt.plot([0, 1], [0, 1], 'k--')
# モデルによって達成された FPR と TPR をプロットする
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# 出力フォルダーに保存されたファイルは、自動的に実験レコードにアップロードされます
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

これで、登録された環境を取得し、それを使用して代替トレーニング スクリプトを実行する新しい実験用の Estimator を構成できるようになりました (デシジョン ツリー分類子はハイパーパラメーターを必要としないため、今回はスクリプト パラメーターはありません)。

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Environment, Experiment
from azureml.widgets import RunDetails

registered_env = Environment.get(ws, 'diabetes-experiment-env')

# トレーニング データセットを取得する
diabetes_ds = ws.datasets.get("diabetes dataset")

# Estimator を作成する
estimator = Estimator(source_directory=experiment_folder,
              inputs=[diabetes_ds.as_named_input('diabetes')], # データセットを入力として渡す
              compute_target = 'local',
              environment_definition = registered_env,
              entry_script='diabetes_training.py')

# 実験を作成する
experiment = Experiment(workspace = ws, name = 'diabetes-training')

# 実験を実行する
run = experiment.submit(config=estimator)
# 実行中に実行の詳細を表示する
RunDetails(run).show()
run.wait_for_completion()

今回は、一致する環境が前回の実行からキャッシュされており、ローカル コンピューティングで再作成する必要がないため、実験の実行速度が速くなります。ただし、異なるコンピューティング ターゲットでも、同じ環境が作成および使用され、実験スクリプトの実行コンテキストの一貫性が確保されます。

実験のメトリックと出力を見てみましょう。

In [ ]:
# 指標の記録を取得する
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

このモデルはロジスティック回帰モデルよりやや優れているように見えるので、登録しましょう。

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Estimator + Environment (Decision Tree)'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## 登録済み環境を表示する

独自の環境を登録するだけでなく、一般的な実験タイプに対して事前構築され「選別された」環境を活用できます。次のコードは、登録されているすべての環境を一覧表示します。

In [ ]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)
for env in envs:
    print("Name",env)

すべての選別された環境には、***AzureML-*** で始まる名前が付いています (このプレフィックスは、独自の環境では使用できません)。

選別された環境を詳しく調べ、それぞれのパッケージに含まれているパッケージを確認しましょう。

In [ ]:
for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

> **詳細情報**: Azure Machine Learning の環境の詳細については、[Azure Machine Learning のドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-use-environments)を参照してください。